<a href="https://colab.research.google.com/github/duridudu/dupository/blob/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0%20chapter%2005_2_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Chapter 05-2 교차검증과 그리드서치**

* Before start : 흠 근데 이렇게 테스트값을 마구 넣고 결정 트리 돌리면 결국 테스트 세트에 잘 맞는 모델이 만들어지는거 아냐? ==> 검증세트의 필요성

###**검증 세트**

테스트세트를 사용하지 않고 모델이 과대/과소적합인지 측정하는 방법. 훈련 세트를 또 나눔!

ex. 약 60%는 훈련세트, 약 20%는 검증세트, 약 20%는 테스트세트

(보통 20~30%를 테스트 세트와 검증 세트로 떼어 놓음. 그러나 훈련 데이터가 아주 많다면 단 몇%만 떼어 놓아도 전체 데이터를 대표하는데 문제가 없음)

* 훈련세트에서 모델을 훈련하고 검증세트로 모델을 평가함. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름. 

  그 다음 이 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련함. 
  
  그리고 마지막에 테스트세트에서 최종 점수를 평가 --> 실전에 투입했을 때 테스트세트의 점수와 비슷한 성능을 기대할 수 있을 것. 



In [6]:
#검증세트 만들기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

#class열을 타깃으로, 나머지 열은 특성 배열에 저장
data=wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

#테스트세트와 훈련세트 나눔.
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

#train_input과 train_target을 다시 훈련세트(sub)와 검증세트(val)로 나눔.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

#훈련, 검증세트로 모델 만들고 평가하기
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
#==> 훈련세트에 과대적합 됨!

0.9971133028626413
0.864423076923077


### **교차 검증**

검증 세트 만드느라 훈련 세트가 줄었다. 그런데 보통 훈련 세트가 많을수록 좋은 모델이 만들어짐.

그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉할 것임. 이럴 때 **교차검증** 사용.

* 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻음. 

* k-폴드 교차 검증 : 훈련 세트를 k겹으로 나누어 교차 검증을 수행하는 것. 보통 5-폴드나 10-폴드를 많이 이용. 이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다. 

* by **cross_validate()** 라는 교차 검증 함수 : 평가할 모델 객체를 첫번째 매개변수로 전달,

 그 다음 앞처럼 직접 검증세트를 떼어내지 않고 훈련세트 전체를 cross_calidate() 함수에 전달함.

* **cross_validate()는 훈련세트를 섞어 폴드를 나누지 않음! 

 우리는 앞서 train_test_split으로 전체 데이터 섞고 훈련세트를 나눴기 때문에 따로 안섞어도 되는데 따로 하려면 분할기(spliter)를 지정해야 함.

 ==> 기본으로 KFold 분할기. 분류 모델일 경우 타깃 틀래스를 골고루 나누기 위해 StraintifedKFold 사용

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores) #최종 점수 아니고 검증 폴드의 점수임!

{'fit_time': array([0.0082581 , 0.00714517, 0.00751281, 0.00729752, 0.00783348]), 'score_time': array([0.00104213, 0.00072932, 0.00086641, 0.00065112, 0.0010314 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
#교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
#분할기
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
#만약 훈련세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음과 같이 작성해야 함.
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores=cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

#이제 교차검증을 알았으니 매개변수를 바꿔가며 테스트를 해보

0.8574181117533719


###**하이파라미터 튜닝**

하이퍼파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터. 매개변수로 표현됨.

* 매개변수가 여러 개이면 하나가 달라지면 다른 것들의 결과도 달라져서 동시에 제어해야 함. 이 때 **그리드 서치** 사용

* by **GridSearchCV 클래스**


In [13]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.00055] } #탐색할 매개변수와 탐색할 값

#탐색 대상 모델과 변수를 전달하여 그리드서치 객체를 만듦
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#==> 이 객체는 결정트리모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행됨. 
#GridSearchCV의 cv 매개변수 기본값은 5. 따라서 min_~값마다 5-폴드 교차검증을 수행. 결국 5*5=25개의 모델을 훈련하는 것. 
#n_jobs 매개변수에서 병렬 실행에 사용할 CPU 코어 수 지정. 매개변수의 기본값은 1
#==> -1로 지정하면 시스템에 있는 모든 코어를 사용함.

#그리드서치 수행
gs.fit(train_input, train_target)

#최적의 하이퍼파라미터 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만, 
#사이킷런의 그리드서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델 훈련
#best_estimator_ 속성에 저장되어 있음. 이 모델을 일반 결정트리처럼 똑같이 사용할 수 있음.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [14]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [15]:
#교차검증으로 얻은 점수
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86607574]
